In [32]:
import numpy as np
import cv2
import os
import scipy
import matplotlib.pylab as plt
import io

In [33]:
params_dir = os.getcwd()+'/camera_params/monocular_camera_params/'
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()
board = cv2.aruco.GridBoard_create(1,1, 0.05, 0.0075, AruCo_dict)

#วาดเส้นแกน
def draw_axis(img, corners, imgpts) :
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255,0,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0,0,255), 10)
    return img

#เขียนtext
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

# 
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    #คำนวณHomography
    h, status = cv2.findHomography(pts_src, pts_dst)

    #วาร์ปภาพต้นทางไปยังปลายทางตาม homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    #เตรียมmaskเพื่อคัดลอกจากwarped_imageลงในเฟรมต้นฉบับ
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    return im_out

In [34]:
vid = cv2.VideoCapture('./left_output-1.avi')
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()
template = cv2.imread('./Template-1.png')
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
qr_img = cv2.imread('./qrc_page-0001.jpg')
im_src_size = qr_img.shape[:2]
src_points = np.float32([[0,0], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ])

while vid.isOpened() :
    ret, frame = vid.read()
    if ret :
        query_img = frame
        adjusted = cv2.convertScaleAbs(query_img, alpha=1.23, beta=75)
        query_gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()

        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        
        if len(good_matches) > 14 :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.35) # H RANSAC

            # get the bounding box around template image
            h, w = template.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (128,0,255), 2, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
            augmented = augmented_image(detected_img, qr_img, src_points, transformed_box)
            
            img = augmented.copy()
            markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(augmented, AruCo_dict, parameters = AruCo_params)
            
            if len(markerCorners) > 0:
                img = cv2.aruco.drawDetectedMarkers(augmented, markerCorners)
                rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.1, K, dist)
                for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                    img = cv2.aruco.drawAxis(augmented, K, dist, rvec, tvec, 0.1)
                    x = tvec[0,0]
                    y = tvec[0,1]
                    z = tvec[0,2]
                    text = "    x={:.1f}\n    y={:.1f}\n    z={:.1f}".format( x, y, z)
                    cX = (corner[0,0][0] + corner[0,2][0])/2
                    cY = (corner[0,0][1] + corner[0,2][1])/2
                    write_text(detected_img, (cX, cY), 20, text)
                ret, brvec, btvec = cv2.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs) 
                if ret :
                    img = cv2.aruco.drawAxis(detected_img, K, dist, brvec, btvec, 0.1)

        cv2.imshow('Video frame',detected_img)


        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()